In [1]:
using BEDFiles, BenchmarkTools

┌ Info: Precompiling BEDFiles [6f44c9a6-7b0d-11e8-3270-d5b481c31be2]
└ @ Base loading.jl:1186


In [2]:
const bf = BEDFile(BEDFiles.datadir("mouse.bed"));

The virtual size of the GWAS data is 1940 observations at each of 10150 SNP positions

In [3]:
size(bf)

(1940, 10150)

The actual size of the memory-mapped matrix of `UInt8` values is 485 rows and 10150 columns

In [4]:
size(bf.data)

(485, 10150)

Because the file is memory-mapped opening the file and accessing the data is fast, even for very large `.bed` files.

In [5]:
@benchmark(BEDFile(BEDFiles.datadir("mouse.bed")))

BenchmarkTools.Trial: 
  memory estimate:  389.42 KiB
  allocs estimate:  61
  --------------
  minimum time:     104.199 μs (0.00% GC)
  median time:      112.663 μs (0.00% GC)
  mean time:        131.108 μs (10.81% GC)
  maximum time:     46.781 ms (99.32% GC)
  --------------
  samples:          10000
  evals/sample:     1

This file, from a study published in 2006, is about 5 Mb in size but data from recent studies, which have samples from tens of
thousands of individuals at over a million SNP positions, would be in the tens or even hundreds of Gb range.

## Raw summaries

Counts of each the four possible values for each column are returned by `counts`.`

In [6]:
counts(bf, dims=1)

4×10150 Array{Int64,2}:
  358   359  252   358    33   359  …    56    56    56    56    56    56
    2     0    4     3     4     1      173   173   162   173   174   175
 1003  1004  888  1004   442  1004      242   242   242   242   242   242
  577   577  796   575  1461   576     1469  1469  1480  1469  1468  1467

Column 2 has no missing values (code `0x01`, the second row in the column-counts table).
In that SNP position for this sample, 359 indivduals are homozygous allele 1 (`G` according to the `.bim` file), 1004 are heterozygous,
and 577 are homozygous allele 2 (`A`).

The counts by column and by row are cached in the `BEDFile` object.
Accesses after the first are extremely fast.

In [7]:
@benchmark(counts($bf, dims=1))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.826 ns (0.00% GC)
  median time:      7.893 ns (0.00% GC)
  mean time:        7.909 ns (0.00% GC)
  maximum time:     13.971 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999


## Instantiating as a count of the second allele

In some operations on GWAS data the data are converted to counts of the second allele, according to

|BEDFile|count   |
|------:|--------:|
| 0x00  | 0       |
| 0x01  | missing |
| 0x10  | 1       |
| 0x11  | 2       |

This can be accomplished by indexing `bedvals` with the `BEDFile` or with a view of the `BEDFile`,
producing an array of type `Union{Missing,Int8}`, which is the preferred way in v0.7 of
representing arrays that may contain missing values.

In [8]:
bedvals[bf]

1940×10150 Array{Union{Missing, Int8},2}:
 1  1  1  1  2  1  2  1  1  1  1  2  1  1  2  …  2         2         2       
 1  1  2  1  1  1  1  2  1  1  1  1  2  2  1     2         2         2       
 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2     2         2         2       
 1  1  1  1  1  1  1  2  1  1  1  1  2  2  1     2         2         2       
 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2     1         1         1       
 1  1  1  1  2  1  2  1  1  1  1  2  1  1  2  …  2         2         2       
 1  1  1  1  2  1  2  1  1  1  1  2  1  1  2     2         2         2       
 1  1  2  1  1  1  1  2  1  1  1  1  2  2  1     2         2         2       
 1  1  2  1  1  1  1  2  1  1  1  1  2  2  1     2         2         2       
 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2     1         1         1       
 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  …  0         0         0       
 1  1  1  1  2  1  2  1  1  1  1  2  1  1  2     2         2         2       
 2  2  2  2  2  2  2  

In [9]:
sort(unique(bedvals[bf]))

4-element Array{Union{Missing, Int8},1}:
 0       
 1       
 2       
  missing

### Summary statistics

The package provides methods for the generics `mean` and `var` from the `Statistics` package.

In [10]:
mean(bf, dims=1)

1×10150 Array{Float64,2}:
 1.113  1.11237  1.28099  1.11203  …  1.8009  1.79966  1.79955  1.79943

In [11]:
var(bf, dims=1)

1×10150 Array{Float64,2}:
 0.469929  0.470089  0.462605  0.469365  …  0.223714  0.223818  0.223923

These methods make use of the cached column or row counts and thus are very fast

In [12]:
@benchmark(mean($bf, dims=1))

BenchmarkTools.Trial: 
  memory estimate:  79.39 KiB
  allocs estimate:  2
  --------------
  minimum time:     39.954 μs (0.00% GC)
  median time:      40.572 μs (0.00% GC)
  mean time:        42.815 μs (4.27% GC)
  maximum time:     626.437 μs (88.90% GC)
  --------------
  samples:          10000
  evals/sample:     1

The column-wise or row-wise standard deviations are returned by `std`.

In [13]:
std(bf, dims=2)

1940×1 Array{Float64,2}:
 0.6504997290784408
 0.6379008244533891
 0.6558172726141286
 0.6532675479248437
 0.6744432174014563
 0.6519092298111158
 0.6779881845456428
 0.6955814098050999
 0.6437566832989493
 0.6505283141088536
 0.665444994623426 
 0.659392039592328 
 0.6641674726999468
 ⋮                 
 0.6599158250006595
 0.688387450736178 
 0.6664063015924304
 0.6613451651895259
 0.6659810347614777
 0.6274577846909379
 0.6823658517777204
 0.6695299551061924
 0.710756592739754 
 0.6387913736114869
 0.6736492722732016
 0.688855476425891 

## Location of the missing values


The positions of the missing data are evaluated by

In [14]:
mp = missingpos(bf)

1940×10150 SparseArrays.SparseMatrixCSC{Bool,Int32} with 33922 stored entries:
  [702  ,     1]  =  true
  [949  ,     1]  =  true
  [914  ,     3]  =  true
  [949  ,     3]  =  true
  [1604 ,     3]  =  true
  [1891 ,     3]  =  true
  [81   ,     4]  =  true
  [990  ,     4]  =  true
  [1882 ,     4]  =  true
  [81   ,     5]  =  true
  [676  ,     5]  =  true
  [990  ,     5]  =  true
  ⋮
  [1791 , 10150]  =  true
  [1795 , 10150]  =  true
  [1846 , 10150]  =  true
  [1848 , 10150]  =  true
  [1851 , 10150]  =  true
  [1853 , 10150]  =  true
  [1860 , 10150]  =  true
  [1873 , 10150]  =  true
  [1886 , 10150]  =  true
  [1894 , 10150]  =  true
  [1897 , 10150]  =  true
  [1939 , 10150]  =  true

In [15]:
@benchmark(missingpos($bf))

BenchmarkTools.Trial: 
  memory estimate:  1.81 MiB
  allocs estimate:  19273
  --------------
  minimum time:     40.182 ms (0.00% GC)
  median time:      41.063 ms (0.00% GC)
  mean time:        42.007 ms (0.33% GC)
  maximum time:     61.587 ms (0.00% GC)
  --------------
  samples:          119
  evals/sample:     1

So, for example, the number of missing data values in each column can be evaluated as

In [16]:
sum(mp, dims=1)

1×10150 Array{Int64,2}:
 2  0  4  3  4  1  4  1  3  3  0  4  0  …  174  173  173  162  173  174  175

although it is faster, but somewhat more obscure, to use

In [17]:
view(counts(bf, dims=1), 2:2, :)

1×10150 view(::Array{Int64,2}, 2:2, :) with eltype Int64:
 2  0  4  3  4  1  4  1  3  3  0  4  0  …  174  173  173  162  173  174  175